In [1]:
import os
import sys
import math
import logging
from pathlib import Path

import numpy as np
import scipy as sp

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
from bhm_at_scale.preprocess import gen_splits, gen_partitions, encode, make_cube
from bhm_at_scale.utils import reorder_cols

To download files from Kaggle register a user first and create a file in your homedirectory `.kaggle/kaggle.json` with content `{"username":"YOURUSERNAME","key":"YOURKEY"}`

In [3]:
import kaggle

kaggle.api.authenticate()
kaggle.api.dataset_download_files('pratyushakar/rossmann-store-sales', path='../data/raw', unzip=True)

## Data cleansing and basic feature engineering

In [6]:
stores

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [7]:
pd.read_csv('../data/raw/train.csv')

/Users/fwilhelm/.miniconda3/envs/bhm-at-scale/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [5]:
stores = pd.read_csv('../data/raw/store.csv')
df = (pd.read_csv('../data/raw/train.csv', infer_datetime_format=True, parse_dates=['Date'])
         .assign(StateHoliday=lambda df: df.StateHoliday.map({'a': 1, 'b': 2, 'c': 3, '0': 0, 0: 0}))
         .merge(stores, how='left', on='Store')
         .sort_values(['Store', 'Date'])
         .assign(Assortment=lambda df: df.Assortment.map({'a': 1, 'b': 2, 'c': 3}))
         .assign(StoreType=lambda df: df.StoreType.map({'a': 1, 'b': 2, 'c': 3, 'd': 4}))
         .assign(StoreId=lambda df: df.groupby(['Store']).ngroup()) # make IDs contiguous
         .assign(Timestep=lambda df: df.groupby(['Store'])['Date'].cumcount())
         .assign(StoreVariant=lambda df: df['StoreType']*10 + df['Assortment'])
         .drop([c for c in stores.columns if c.startswith('Competition')], axis=1)
         .drop([c for c in stores.columns if c.startswith('Promo2S')], axis=1)
         .drop(['PromoInterval', 'StoreType', 'Assortment'], axis=1)
     )

/Users/fwilhelm/.miniconda3/envs/bhm-at-scale/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
# Use this to split stores up into 'virtual' smaller stores just to demonstrate the scalability of this approach
# df = (df.groupby('Store', as_index=False).apply(lambda df: df.assign(part=gen_partitions(len(df), 7, 22)))
#         .reset_index(drop=True)
#         .query('part >= 0')
#         .assign(Store=lambda x: x.Store*1000+x.part)
#         .drop(['part'], axis=1)
#         .sort_values(['StoreId', 'Date'])
#         .reset_index(drop=True))

## Dummy encoding of categorical values

In [9]:
edf = encode(df.drop(columns='Customers'), ['DayOfWeek', 'StoreVariant', 'StateHoliday'], drop_first=False)
edf = edf.drop(columns=['Date', 'Store'])
edf.loc[edf['Open'] == 0, 'Sales'] = np.nan
edf = edf.drop(columns='Open')  # Keep the open column

In [10]:
edf = reorder_cols(edf, first=['StoreId', 'Timestep'], last='Sales')

In [11]:
edf.to_csv('../data/preprocessed/edf.csv', index=False)

## Generate a multi-dimensional array for input

In [12]:
X = make_cube(edf, ['StoreId', 'Timestep'])

In [13]:
X_train, X_test = X[:1000, ...], X[1000:, ...]

In [14]:
np.savez('../data/preprocessed/X_train.npz', X_train)
np.savez('../data/preprocessed/X_test.npz', X_test)

In [15]:
df.to_csv('../data/result/df.csv', index=False)